In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("final_data.csv")
data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
print(data_matrix.shape)


(7163, 1590)


In [4]:
delta=df[['u_id', 'a_id', 'score']]
delta

,u_id,a_id,score
0,5,28891,8
1,5,32935,9
2,5,37141,8
3,5,8074,7
4,5,7054,7
...,...,...,...
59162,15121,3230,8
59163,15698,3230,8
59164,22915,3230,7
59165,24855,3230,6


In [5]:
user_id_to_index = {} #helper function
for idx in range(len(data_matrix.index)):
    u_id = data_matrix.index[idx]
    user_id_to_index[u_id] = idx
    
user_id_to_index[5]

4

In [6]:
anime_id_to_index = {} #helper function
for idx in range(len(data_matrix.columns)):
    a_id = data_matrix.columns[idx]
    anime_id_to_index[a_id] = idx

anime_id_to_index[35760]

1413

In [8]:
user_id_to_index[1892]

1000

In [ ]:
def spectral_regularization_model(x):

    delta = df[['u_id', 'a_id', 'score']] #set of all known ratings
    num_users = data_matrix.shape[0]
    num_anime = data_matrix.shape[1]

    R=cp.Variable(shape=(num_users,num_anime)) #predicted rating matrix
    

    training_error=[]
    for ui in range(len(delta)):
        u=user_id_to_index[int(delta.at[ui,'u_id'])]
        i=anime_id_to_index[int(delta.at[ui,'a_id'])]
        training_error.append((R[u][i]-int(delta.at[ui,'score']))**2) #(R_ui-A_ui)^2

    training_error_sum=cp.sum(training_error) #sum((R_ui-A_ui)^2)


    obj=cp.Minimize(training_error_sum+ x*cp.norm(R,"nuc"))
    prob=cp.Problem(obj)
    prob.solve()
    print("status:", prob.status)
    
    R = np.zeros((num_users, num_anime))
    for u in range(num_users):
        for i in range(num_anime):
            R[u, i] = avg_rating + b_u.value[u] + b_i.value[i]
    
    print("status:", prob.status)

    



In [9]:
spectral_regularization_model(10000)


C:\Users\pakch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


KeyboardInterrupt: 